In [10]:
import os, duckdb, pandas as pd, ast

PAI_DIR = os.path.dirname(os.getcwd())  # diretório atual para notebooks jupyter
DATA_RAW = os.path.join(PAI_DIR, "data", "raw")
DATA_PROCESSED = os.path.join(PAI_DIR, "data", "processed")

In [17]:
df = pd.read_excel(os.path.join(DATA_RAW, "estimativa_cre_12-08-25.xlsx"), sheet_name='Projeção', header=8)

df.head(10)

,Ordem,UG ajustada,Descrição UG,Nova Fonte,Descrição Fonte,Classificação ajustada,Classificação p/ intra,Nomenclatura de receita,REC_Grupo,REC_Subgrupo,...,2025-05-01 00:00:00,2025-06-01 00:00:00,2025-07-01 00:00:00,2025-08-01 00:00:00,2025-09-01 00:00:00,2025-10-01 00:00:00,2025-11-01 00:00:00,2025-12-01 00:00:00,X,2025
0,1.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125101.0,NaN,Imposto sobre a Propriedade de Veículos Automo...,Receita Bruta,Receita Estimada,...,60103862.04,55451372.88,67012183.85,5.178575e+07,4.074573e+07,3.678900e+07,2.007564e+07,1.688792e+07,NaN,5.896801e+08
1,2.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125101.0,NaN,RENÚNCIA POTENCIAL - Imposto sobre a Proprieda...,Deduções da Receita,Renúncia Potencial,...,0.00,0.00,0.00,-3.170340e+05,-3.170340e+05,-3.170340e+05,-3.170340e+05,-3.170340e+05,NaN,-1.585170e+06
2,3.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125102.0,NaN,Imposto sobre a Propriedade de Veículos Automo...,Receita Bruta,Receita Estimada,...,2138059.52,2204966.41,2445689.25,2.123463e+06,1.684594e+06,1.631723e+06,1.654503e+06,2.277831e+06,NaN,2.180183e+07
3,4.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125103.0,NaN,Imposto sobre a Propriedade de Veículos Automo...,Receita Bruta,Receita Estimada,...,1636492.21,1512230.86,3555552.73,1.671997e+06,1.508568e+06,1.349283e+06,1.027388e+06,1.150734e+06,NaN,2.070471e+07
4,5.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125104.0,NaN,Imposto sobre a Propriedade de Veículos Automo...,Receita Bruta,Receita Estimada,...,743283.70,715282.10,1386125.66,4.829032e+05,3.515778e+05,3.242022e+05,3.771643e+05,4.141241e+05,NaN,8.018219e+06
5,6.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125201.0,NaN,Imposto sobre Transmissão “Causa Mortis” e Doa...,Receita Bruta,Receita Estimada,...,2383295.45,3019934.54,2916640.05,3.065803e+06,3.055627e+06,3.044421e+06,3.059357e+06,3.038858e+06,NaN,3.633457e+07
6,7.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125201.0,NaN,Imposto sobre Transmissão “Causa Mortis” e Doa...,Deduções da Receita,Renúncia Potencial,...,0.00,0.00,0.00,-8.876000e+05,-8.876000e+05,-8.876000e+05,-8.876000e+05,-8.876000e+05,NaN,-4.438000e+06
7,8.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125202.0,NaN,Imposto sobre Transmissão “Causa Mortis” e Doa...,Receita Bruta,Receita Estimada,...,72943.78,287653.30,79990.16,1.198962e+05,1.223151e+05,1.248216e+05,1.268455e+05,1.287287e+05,NaN,1.588250e+06
8,9.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125203.0,NaN,ITCMD Dívida Ativa - Estado,Receita Bruta,Receita Estimada,...,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
9,10.0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125204.0,NaN,ITCMD Dívida Ativa - Multas e Juros de Mora d,Receita Bruta,Receita Estimada,...,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00


In [19]:
df.columns

Index([                  'Ordem',             'UG ajustada',
                  'Descrição UG',              'Nova Fonte',
               'Descrição Fonte',  'Classificação ajustada',
        'Classificação p/ intra', 'Nomenclatura de receita',
                     'REC_Grupo',            'REC_Subgrupo',
              'Método principal',       2025-01-01 00:00:00,
             2025-02-01 00:00:00,       2025-03-01 00:00:00,
             2025-04-01 00:00:00,       2025-05-01 00:00:00,
             2025-06-01 00:00:00,       2025-07-01 00:00:00,
             2025-08-01 00:00:00,       2025-09-01 00:00:00,
             2025-10-01 00:00:00,       2025-11-01 00:00:00,
             2025-12-01 00:00:00,                       'X',
                            2025],
      dtype='object')

In [20]:
df = df.iloc[:, :-2]

date_col = df.iloc[:,11:]
id_vars = df.iloc[:,:11].columns.tolist()

In [23]:
id_vars

['Ordem',
 'UG ajustada',
 'Descrição UG',
 'Nova Fonte',
 'Descrição Fonte',
 'Classificação ajustada',
 'Classificação p/ intra',
 'Nomenclatura de receita',
 'REC_Grupo',
 'REC_Subgrupo',
 'Método principal']

In [24]:
df_long = pd.melt(df, id_vars=id_vars, value_vars=date_col, var_name='data', value_name='valor')
df_long['data'] = pd.to_datetime(df_long['data'])
df_long['mes'] = df_long['data'].dt.month
df_long['ano'] = df_long['data'].dt.year


df_long = df_long.drop(columns=['data'])

colunas_ordenadas = id_vars + ['ano', 'mes', 'valor']

df_final = df_long[colunas_ordenadas]

df_final = df_final.drop(columns=['Ordem'])

df_final['valor'] = df_final['valor'].fillna(0)

colunas_renomear = {
    "Ordem": "ordem",
    "UG ajustada": "ug_codigo",
    "Descrição UG": "descricao_ug",
    "Nova Fonte": "fonte_recurso",
    "Descrição Fonte": "descricao_fonte",
    "Classificação ajustada": "classificacao_ajustada",
    "Classificação p/ intra": "classificacao_intra",
    "Nomenclatura de receita": "nomenclatura_receita",
    "REC_Grupo": "rec_grupo",
    "REC_Subgrupo": "rec_subgrupo",
    "Método principal": "metodo_principal"
}

# Renomeando
df_final = df_final.rename(columns=colunas_renomear)

df_final['valor'] = pd.to_numeric(df_final['valor'], errors='coerce')

df_final['ug_codigo'] = df_final['ug_codigo'].astype(str)

df_final.to_parquet(os.path.join(DATA_PROCESSED, 'estimativa_cre_12-08-25.parquet'), index=False, engine='pyarrow', compression='snappy')